In [ ]:
!pip install ultralytics

In [ ]:
%matplotlib inline


In [ ]:
!pip install --upgrade Pillow
!pip install matplotlib
!pip install numpy


In [ ]:
!unzip "/content/test_image_left.zip"

In [ ]:
!unzip "/content/train_iamge_left.zip"

In [ ]:
torch.cuda.get_device_name(0)

# YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks



YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.2/78.2 GB disk)


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('/content/drive/MyDrive/CSC420_car_detection/image_left/um_000004.jpg')
cv2_imshow(img)

In [ ]:
!pip install yolov5


In [ ]:
from yolov5 import YOLOv5

model_path = "yolov5s.pt"
yolov5_model = YOLOv5(model_path)


In [ ]:
from PIL import Image
import numpy as np
import cv2

image_path = '/content/drive/MyDrive/CSC420_car_detection/image_left/um_000004.jpg'
yolov5_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
yolov5_image = np.array(yolov5_image)


In [ ]:
yolov5_results = yolov5_model.predict(yolov5_image, size=640)


In [ ]:
car_class_id = 2

# Filter results for cars
yolov5_car_detections = yolov5_results.xyxy[0][yolov5_results.xyxy[0][:, 5] == car_class_id]


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Display the image with detections
plt.figure(figsize=(10, 10))
plt.imshow(yolov5_image)
ax = plt.gca()

for *xyxy, conf, cls in yolov5_car_detections:
    x1, y1, x2, y2 = map(int, xyxy)
    label = f'Car {conf:.2f}'
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linewidth=2)
    ax.add_patch(rect)
    # change the label position: vertical and horizontal alignment
    ax.text(x1, y1-10, label, color='white', fontsize=8,
            verticalalignment='top', horizontalalignment='left',
            bbox=dict(facecolor='red', alpha=0.5, edgecolor='none', boxstyle='round,pad=0.1'))

plt.axis('off')
plt.show()


# Testing on an Image


In [ ]:
# single image inference
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source '/content/drive/MyDrive/CSC420_car_detection/image_left/um_000004.jpg'

In [ ]:
display.Image(filename='/content/yolov5/runs/detect/exp/um_000004.jpg', width=600)

# Testing on a Video

In [ ]:
# video inference
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source '/content/drive/MyDrive/CSC420_car_detection/test_image_left.mp4'

In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = '/content/yolov5/runs/detect/exp/output.mp4'

# Compress and display the video inline
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [ ]:
# test the model

model = YOLO('run/detect/train/weights/best.pt')

results = model('test_images/000018.png', save=True)
# results = model('test_videos/video_1.mp4', save=True)

# YOLO v8


In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


In [ ]:
yolov8_MODEL_PATH = "yolov8x.pt"  # Ensure the model file is in the working directory or provide the full path
yolov8_model = YOLO(yolov8_MODEL_PATH)
yolov8_model.fuse()  # Optional: Fusing the model can optimize its performance


In [ ]:
yolov8_image_path = '/content/drive/MyDrive/CSC420_car_detection/image_left/um_000004.jpg'
yolov8_image = cv2.imread(yolov8_image_path, cv2.IMREAD_COLOR)
yolov8_image = np.array(yolov8_image)

# Perform detection
yolov8_results = yolov8_model(yolov8_image)


In [ ]:
car_class_id = 2  # Change this based on the specific class ID for cars in your model

# Extract car detections
yolov8_car_detections = yolov8_results[0].boxes[yolov8_results[0].boxes.cls == car_class_id]


In [ ]:
# Convert detections to format suitable for display
import matplotlib.patches as patches

boxes = yolov8_car_detections.xyxy.cpu().numpy()
confidences = yolov8_car_detections.conf.cpu().numpy()

plt.figure(figsize=(12, 8))
plt.imshow(yolov8_image)
ax = plt.gca()

for box, confidence in zip(boxes, confidences):
    x1, y1, x2, y2 = box.astype(int)
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='red', linewidth=2)
    ax.add_patch(rect)
    # Label positioned above the bounding box and background made semi-transparent
    label = f'Car {confidence:.2f}'
    ax.text(x1, y1-10, label, color='white', fontsize=8,
            verticalalignment='top', horizontalalignment='left',
            bbox=dict(facecolor='red', alpha=0.5, edgecolor='none', boxstyle='round,pad=0.1'))

plt.axis('off')
plt.show()

In [ ]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Open the video file
video_path = '/content/drive/MyDrive/CSC420_car_detection/test_image_left-ezgif.com-video-speed.mp4'
cap = cv2.VideoCapture(video_path)


# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Define the codec and create VideoWriter object to save the output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 'mp4v' for .mp4 output
out = cv2.VideoWriter('yolov5_test_image_left_output_video.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Process the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Convert frame to RGB from BGR (cv2 default)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform detection
    results = model(frame_rgb)
    car_detections = results.xyxy[0]  # First tensor [xyxy, conf, cls]

    # Iterate through detections and draw boxes
    for *xyxy, conf, cls in car_detections:
        if cls == 2:  # Class '2' for cars
            x1, y1, x2, y2 = map(int, xyxy)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'Car: {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Save the frame with detections to the output video
    out.write(frame)

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()

# Car Detection + Viewpoint Classification (q8)

In [ ]:
!unzip "/content/saved_models-20240419T063014Z-001.zip"

In [ ]:
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import cv2
import os
import io

# Define the folder containing the images
folder_path = '/content/image_left'
output_folder_path = '/content/yolov8_viewpoint'
os.makedirs(output_folder_path, exist_ok=True)

# Load the car detection YOLOv8 model
yolov8_model_path = "yolov8x.pt"
yolov8_model = YOLO(yolov8_model_path)

# Load the viewpoint classification model
viewpoint_model_path = '/content/saved_models/ResNet50_model_exp2.h5'
viewpoint_model = load_model(viewpoint_model_path)

# Class to angle mapping for viewpoint classification
class_to_angle = {
    0: 0, 1: 120, 2: 150, 3: 180, 4: 210, 5: 240,
    6: 270, 7: 300, 8: 30, 9: 330, 10: 60, 11: 90
}

# Function to plot an arrow for viewpoint
def plot_arrow(angle, car_box, ax):
    radians = math.radians(angle-90)  # Adjust arrow direction
    arrow_start = (car_box[0] + car_box[2]) // 2, car_box[3]  # Arrow starts at the bottom center of the car box
    arrow_length = (car_box[2] - car_box[0]) // 2  # Arrow length is half the car box width
    x_offset = arrow_length * math.cos(radians)
    y_offset = arrow_length * math.sin(radians)
    ax.annotate('', xy=arrow_start, xytext=(arrow_start[0] + x_offset, arrow_start[1] + y_offset),
                arrowprops=dict(facecolor='red', shrink=0.05, width=1, headwidth=8),
                horizontalalignment='center', verticalalignment='center')

# # Read the test image
# test_image_path = '/content/image_left/um_000004.jpg'  # Update this path
# test_image = cv2.imread(test_image_path)
# test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

# # Perform car detection
# yolov8_results = yolov8_model(test_image)
# car_detections = yolov8_results[0].boxes[yolov8_results[0].boxes.cls == 2]  # Assuming class 2 is 'car'

# # Plot original image with bounding boxes
# fig, ax = plt.subplots(figsize=(12, 8))
# ax.imshow(test_image)
# ax.axis('off')  # Hide the axes

# # Iterate over detected cars
# for box in car_detections.xyxy.cpu().numpy():
#     x1, y1, x2, y2 = box.astype(int)
#     # Crop the car image
#     car_image = test_image[y1:y2, x1:x2]
#     car_image = cv2.resize(car_image, (224, 224))
#     car_image_array = img_to_array(car_image)
#     car_image_array = np.expand_dims(car_image_array, axis=0)

#     # Predict the viewpoint
#     prediction = viewpoint_model.predict(car_image_array)
#     predicted_class = np.argmax(prediction)
#     predicted_angle = class_to_angle[predicted_class]
#     print(predicted_angle)

#     # Draw bounding box and viewpoint arrow
#     rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='red', facecolor='none')
#     ax.add_patch(rect)
#     plot_arrow(predicted_angle, (x1, y1, x2, y2), ax)

# plt.show()

for image_name in os.listdir(folder_path):
    # Check if file is an image
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
        test_image_path = os.path.join(folder_path, image_name)
        test_image = cv2.imread(test_image_path)
        test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

        # Perform car detection
        yolov8_results = yolov8_model(test_image)
        car_detections = yolov8_results[0].boxes[yolov8_results[0].boxes.cls == 2]  # Assuming class 2 is 'car'

        # Plot original image with bounding boxes
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.imshow(test_image)
        ax.axis('off')  # Hide the axes

        # Iterate over detected cars
        for box in car_detections.xyxy.cpu().numpy():
            x1, y1, x2, y2 = box.astype(int)
            # Crop the car image
            car_image = test_image[y1:y2, x1:x2]
            car_image = cv2.resize(car_image, (224, 224))
            car_image_array = img_to_array(car_image)
            car_image_array = np.expand_dims(car_image_array, axis=0)

            # Predict the viewpoint
            prediction = viewpoint_model.predict(car_image_array)
            predicted_class = np.argmax(prediction)
            predicted_angle = class_to_angle[predicted_class]

            # Draw bounding box and viewpoint arrow
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            plot_arrow(predicted_angle, (x1, y1, x2, y2), ax)

        plt.show()
        # Save the figure
        output_image_name = os.path.splitext(image_name)[0] + '.png'  # Save as PNG
        output_image_path = os.path.join(output_folder_path, output_image_name)

        plt.savefig(output_image_path, bbox_inches='tight')
        plt.close(fig)  # Close the figure to free up memory
